# Asteroid Light Curve Examples - Part 1

This notebook contains examples deep learning techniques applied to the asteroid light curve data from http://alcdef.org.


# Objectives
- Understand when a convolutional neural network (CNN) might be applicable.
- See how to apply a 1D-CNN to time-series data.
- See how to build a more complex model that takes both time-series and categorical inputs.

# Parameters

In [ ]:
# Path to the ALCDEF_ALL dataset downloaded from http://alcdef.org
# Download the full archive as a .zip file. Extract its contents to this
# directory. It should be ~14K .txt files.
data_dir = 'data/ALCDEF_ALL'

# Discard any light curves with fewer than this many samples
min_samples = 100

# Resample light curves to common number of samples
nb_samples = 100

# Discard any light curve that isn't among the nb_classes most common objects
nb_classes = 20

# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from glob import glob
from scipy.signal import resample
from collections import Counter

import random

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Input

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import pandas as pd
import seaborn as sns

from ml4ssa_utils import visualize_embedding, load_alcdef_data, plot_alcdef_examples, normalize_features, plot_confusion_matrix

# Load Dataset

Load data from Astroid Lightcurve Photometry Database (http://alcdef.org/)

In [ ]:
data = load_alcdef_data(
    data_dir=data_dir,
    min_samples=min_samples,
    resample_to=nb_samples,
    reduce_to_top=nb_classes
)

In [ ]:
# Gather a list of the object names we'll be working with
names = list(set([item['OBJECTNAME'] for item in data]))

# Visualize Examples

In [ ]:
plot_alcdef_examples(data)

# Generate Train and Test Data Sets

In [ ]:
X = np.array([ item['DATA_RESAMPLED'][:,1] for item in data ])
y = np.array([ names.index(item['OBJECTNAME']) for item in data ])

# Reserve 20% of the data for testing
# Startify the data split so that the train and test sets have the same class distribution
X_train, X_test, y_train, y_test, data_train, data_test = train_test_split(X, y, data, test_size=0.20, stratify=y)

print('Generated train and test sets with the following sizes:')
print('Train X (features) {}, y (targets) {}'.format(X_train.shape, y_train.shape))
print('Test  X (features) {}, y (targets) {}'.format(X_test.shape, y_test.shape))

## Review Class Distributions to Understand Performance of Random Classifier

It's always helpful to understand how well a random classifier should perform. This sets a worst case baseline. If you're doing better than this performance, you know at least something is working. If your classifier is performing worse than random, something is broken. If it's performing at the same level as random, it's either broken or you have a very hard problem (at least with your current size and distribution of training data).

In [ ]:
class_counts = np.sum(to_categorical(y_test), axis=0)
class_proportions = class_counts / np.sum(class_counts)
max_proportion = np.max(class_proportions)
random_performance = 1./nb_classes

print('Random Performance: {:.3f}'.format(1./nb_classes))
print('Mode Collapse Performance: {:.3f}'.format(max_proportion))
print('-'*65)
for name, proportion in zip(names, class_proportions):
    print('{:15} {:.3f} {}'.format(name, proportion, 'largest' if proportion == max_proportion else ''))

# Try an MLP (multi-layer perceptron) similiar to the TLE Example

In [ ]:
metric='acc'

nb_classes = len(names)

model = Sequential()
model.add(Dense(units=100,activation='relu', input_shape=(100,)))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=nb_classes, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=[metric]
)

# keras is complaining that I need to evaluate the model before printing a summary
# model.predict(np.zeros((16,9)))

model.summary()

In [ ]:
plot_confusion_matrix(model, X_test, y_test, 'Untrained MLP on Test Data')

In [ ]:
# Prepare data to pass to model
repeats = 100
# The repeats value here is used to artifically increase the size of our training set.
# This forces keras to treat <repeats> passes through the training set as a single epoch and we
# get to avoid a huge number of progress bars and short-term variance in metrics.
train_features = normalize_features(X_train.repeat(repeats, axis=0))
train_targets = to_categorical(y_train.repeat(repeats, axis=0))
test_features = normalize_features(X_test)
test_targets = to_categorical(y_test)

# Fit model to data
model.fit(
    train_features, train_targets,
    validation_data=(test_features, test_targets),
    epochs=10,
    batch_size=16,
    callbacks=[EarlyStopping(patience=3, monitor='val_loss')],
    verbose=1
)

In [ ]:
plot_confusion_matrix(model, X_test, y_test, 'Test Data')

In [ ]:
plot_confusion_matrix(model, X_train, y_train, 'Train Data')

# CNN Model

Now that we have two baselines (random performance and the MLP we used for TLE data), let's look at improving our performance with a different model.

In [ ]:
metric='acc'

nb_classes = len(names)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters=16, kernel_size=5, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=nb_classes, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=[metric]
)

# keras is complaining that I need to evaluate the model before printing a summary
model.predict(np.zeros((16,100,1)))

model.summary()

In [ ]:
# Prepare data to pass to model
repeats = 100
# The repeats value here is used to artifically increase the size of our training set.
# This forces keras to treat <repeats> passes through the training set as a single epoch and we
# get to avoid a huge number of progress bars and short-term variance in metrics.
train_features = normalize_features(X_train.repeat(repeats, axis=0))
train_targets = to_categorical(y_train.repeat(repeats, axis=0))
test_features = normalize_features(X_test)
test_targets = to_categorical(y_test)

# The convolutional layers will expect a "channels" dimension at the end.
train_features = np.expand_dims(train_features, axis=-1)
test_features = np.expand_dims(test_features, axis=-1)

model.fit(
    train_features, train_targets,
    validation_data=(test_features, test_targets),
    epochs=10,
    batch_size=16,
    callbacks=[EarlyStopping(patience=5, monitor='val_loss')],
    verbose=1
)

# Light Curve Embedding Based on Extracted Features

In [ ]:
# Here we extract the intermediate features/activations from the layer named penultimate
layer_name = 'penultimate'
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
X_penultimate_test = intermediate_layer_model.predict(test_features)

In [ ]:
visualize_embedding(X_penultimate_test, y_test)

In [ ]:
plot_confusion_matrix(model, X_test, y_test, '')

# Multi-Modal Input

The above models use features from single modality (sampled light curves). In real world problems, we often have multiple data types that will be relevant to our problem. For exampoe, we typically at least have metadata associated with sampled data.

The convolutional layers were motivated by the assumption that our sampled data was translationally invariant. As we have no reason to believe this should be the case for our metadata (it's not even clear what that would mean), we'll need to think about how best to incorporate additional data.

In [ ]:
def generate_metadata_vector(item):
    '''Generate a metadata vector of the form <one-hot-encoded filter value> | <phase>.
    
    The data appears to have 3 different filter codes and a single phase value so the metadata vector
    will be of length 4.
    '''
    v = np.zeros(4)
    filter_codes = ['V', 'R', 'C']
    filter_ndx = filter_codes.index(item['FILTER'])
    v[filter_ndx] = 1
    v[-1] = float(item['PHASE']) / 60. # 60 was chosen as it was the largest value observed in a chunk of the data
    return v

In [ ]:
metric='acc'

nb_classes = len(names)

nb_metadata_inputs = 4

cnn_input = Input(shape=(nb_samples,1), name='cnn_input')
x = Conv1D(filters=64, kernel_size=5, activation='relu')(cnn_input)
x = MaxPooling1D()(x)
x = Conv1D(filters=32, kernel_size=5, activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(filters=16, kernel_size=5, activation='relu')(x)
cnn_output = GlobalAveragePooling1D()(x)

metadata_input = Input(shape=(nb_metadata_inputs,), name='metadata_input')
x = Dense(units=10, activation='relu')(metadata_input)
metadata_output = Dense(units=10, activation='relu')(x)

merged = keras.layers.concatenate([cnn_output, metadata_output])

final_output = Dense(units=nb_classes, activation='softmax')(merged)

model = Model([cnn_input, metadata_input], final_output)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=[metric]
)

model.summary()

In [ ]:
# Prepare data to pass to model
repeats = 100
# The repeats value here is used to artifically increase the size of our training set.
# This forces keras to treat <repeats> passes through the training set as a single epoch and we
# get to avoid a huge number of progress bars and short-term variance in metrics.
train_features = normalize_features(X_train.repeat(repeats, axis=0))
train_targets = to_categorical(y_train.repeat(repeats, axis=0))
test_features = normalize_features(X_test)
test_targets = to_categorical(y_test)

# The convolutional layers will expect a "channels" dimension at the end.
train_features = np.expand_dims(train_features, axis=-1)
test_features = np.expand_dims(test_features, axis=-1)

# TODO: Fix this to use actual metadata features
train_metadata_features = np.stack([ generate_metadata_vector(item) for item in data_train ]).repeat(repeats, axis=0)
test_metadata_features = np.stack([ generate_metadata_vector(item) for item in data_test ])

model.fit(
    [train_features, train_metadata_features], train_targets,
    validation_data=([test_features, test_metadata_features], test_targets),
    epochs=10,
    batch_size=16,
    callbacks=[EarlyStopping(patience=5, monitor='val_loss')],
    verbose=1
)